In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from io import BytesIO
from zipfile import ZipFile
from urllib.request import urlopen

Reading in MSP data, using ZULU time variable to create a date and time stamp

In [2]:
HTM_url = urlopen("https://www.hackthemachine.ai/s/HTM_MSP_Finalcsv.zip")

zipfile = ZipFile(BytesIO(HTM_url.read()))

msp_data = pd.read_csv(zipfile.open('HTM_MSP_Final.csv'), parse_dates=True)

msp_data.rename(columns = {'AIRCRAFT': 'Aircraft'}, inplace = True)

msp_data['Fault Date'] = pd.to_datetime(msp_data['ZULU_TIME'].str.split(" ", n = 2, expand = True)[1])

In [3]:
msp_data.head()

,Aircraft,SQUADRON,LOT,MSP,ZULU_TIME,FLIGHT_MODE,Fault Date
0,1,HH,100,ZJMZTL,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01
1,1,HH,100,JDJDMQ,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01
2,1,HH,100,RTVBHP,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01
3,1,HH,100,RTUXHP,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01
4,1,HH,100,RTUXMZ,01-APR-2014 00:25:39:00,EngineTurn,2014-04-01


Reading in MAF data, converting recevied and completed dates to datetime objects

In [4]:
HTM_url = urlopen("https://www.hackthemachine.ai/s/HtM_MAF-Data_Finalcsv.zip")

zipfile = ZipFile(BytesIO(HTM_url.read()))

maf_data = pd.read_csv(zipfile.open('HtM_MAF Data_Final.csv'), parse_dates=['Received Date', 'Completion Date'])

In [5]:
maf_data.head()

,Job Code,Aircraft,Transaction Code,Malfunction Code,Action Taken Code,Description of Problem,Correction of Problem,Received Date,Completion Date,Corrosion,Bare Metal,Corrosion Prevention Treatment,Routine Maintenance,Unscheduled Maintenance,Mission-Related Maintenance,Failure
0,0NGHY44WC8118573,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN
1,0NGHY45WC8118591,42,11,0,N,Perform system or component checks,Completed the component or system test,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN
2,0NGHY46WC8118581,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN
3,0NGHY47WC8118577,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN
4,0NGHY48WC8118586,42,11,0,N,Perform a periodic inspection,Completed the inspection,2012-04-26,2014-02-27,NaN,NaN,NaN,Yes,NaN,NaN,NaN


Drop rountine mainentance

In [6]:
maf_data = maf_data[maf_data['Routine Maintenance'] != 'Yes']

Adding field that indicates whether repair pertained to corrosion, grouping by aircraft and received date

In [7]:
maf_data['corr_action'] = maf_data.groupby(['Aircraft', 'Received Date'])['Corrosion'].transform(lambda x: any(x == 'Yes'))

In [8]:
maf_data[['Aircraft', 'Received Date', 'corr_action']].drop_duplicates()['corr_action'].value_counts()

False    41521
True      2625
Name: corr_action, dtype: int64

based on the value counts of the corr_action variable, we have a highly imbalanced dataset.  We will use upsampling/downsampling and adjust our evaluation metric (precison, F1) to deal with this imbalance

Adding msp frequency to repair data

In [9]:
maf_corr = maf_data[['Aircraft', 'Received Date', 'Completion Date', 'corr_action']].drop_duplicates()

maf_corr.head()

,Aircraft,Received Date,Completion Date,corr_action
19,42,2012-04-26,2014-02-27,False
31,42,2012-04-26,2014-02-25,False
57,43,2012-09-13,2015-06-05,False
70,43,2012-09-13,2015-04-30,False
80,43,2012-09-14,2015-06-05,False


For each Aircraft repair, we will add the frequncy of the MSP codes in the 30 days prior

In [20]:
msp_corr_pre = []

for repair in maf_corr.iterrows():
    
    # matching aircraft
    
    msp_subset = msp_data[['Aircraft', 'MSP', 'Fault Date']][msp_data['Aircraft'] == repair[1][0]]
    
    # filtering out MSP codes outside of 30-day window prior to repair
    
    msp_subset = msp_subset[(repair[1][1] - msp_subset['Fault Date'] < np.timedelta64(30, 'D')) & (repair[1][1] - msp_subset['Fault Date'] > np.timedelta64(0, 'D'))]
    
    # adding repair date and corrosion flag
    
    msp_subset['Repair Date'] = repair[1][1]
    
    msp_subset['Corrosion'] = repair[1][3]
    
    # counting freq of each MSP code
    
    msp_counts = msp_subset.groupby(['Aircraft', 'Repair Date', 'MSP', 'Corrosion']).size().reset_index(name='freq')
    
    msp_corr_pre.append(msp_counts)

    
msp_corr_pre = pd.concat(msp_corr_pre)


In [14]:
msp_corr_pre.head()

,Aircraft,Repair Date,MSP,Corrosion,freq
0,42,4/26/12,FVFVJD,0,5
1,42,4/26/12,FVFVZJ,0,18
2,42,4/26/12,FVHPHP,0,111
3,42,4/26/12,FVHPMZ,0,2
4,42,4/26/12,FVHPZJ,0,2


Saving pre frequencies to CSV

In [15]:
msp_corr_pre.to_csv('../../HTM_data/msp_freq_pre.csv', index = False)

Convert Corrosion to T/F

In [30]:
msp_corr_pre['Corrosion'] = msp_corr_pre['Corrosion'] == 1.0

Convert Aircraft to Category

In [31]:
msp_corr_pre['Aircraft'] = msp_corr_pre['Aircraft'].astype('category')

Adding pre marker to frequency

In [32]:
msp_corr_pre.rename(columns = {'freq': 'pre_freq'}, inplace = True)

Based on SME input, the MSP codes we should focus on when building a predictive model typically stop appearing when an action for corrosion has been taken.  Therefore we will estimate mean frequncy before and after a corrosion repair for all MSP codes, and then focus the top 20 MSP codes with the largest decrease post-maintenance action.

First we will gather the frequncey of MSP codes 30-days post corrosion maintenance action, as we have already done so for the 30-day prio window.

In [19]:
msp_corr_post = []

for repair in maf_corr.iterrows():
    
    # matching aircraft
    
    msp_subset = msp_data[['Aircraft', 'MSP', 'Fault Date']][msp_data['Aircraft'] == repair[1][0]]
    
    # filtering out MSP codes outside of 30-day window after Completion
    
    msp_subset = msp_subset[(repair[1][2] - msp_subset['Fault Date'] < np.timedelta64(30, 'D')) & (repair[1][2] - msp_subset['Fault Date'] < np.timedelta64(0, 'D'))]
    
    # adding repair date and corrosion flag
    
    msp_subset['Repair Date'] = repair[1][1]
    
    msp_subset['Corrosion'] = repair[1][3]
    
    # counting freq of each MSP code
    
    msp_counts = msp_subset.groupby(['Aircraft', 'Repair Date', 'MSP', 'Corrosion']).size().reset_index(name='freq')
    
    msp_corr_post.append(msp_counts)

    
msp_corr_post = pd.concat(msp_corr_post)

Saving post frequncies to CSV

In [20]:
msp_corr_post.to_csv('../../HTM_data/msp_freq_post.csv', index = False)

Convert Corrosion to T/F

In [23]:
msp_corr_post['Corrosion'] = msp_corr_post['Corrosion'] == 1.0

Convert Aircraft to Category

In [24]:
msp_corr_post['Aircraft'] = msp_corr_post['Aircraft'].astype('category')

Adding post marker to frequency

In [25]:
msp_corr_post.rename(columns = {'freq': 'post_freq'}, inplace = True)

Merging pre/post frequencies

In [39]:
msp_pre_post = msp_corr_post.merge(msp_corr_pre, on = ['Aircraft', 'Repair Date', 'MSP', 'Corrosion'])

Saving pre/post frequncies to CSV

In [40]:
msp_pre_post.to_csv('../../HTM_data/msp_pre_post.csv', index = False)

Estimating change in frequency after repair

In [42]:
msp_pre_post['per_change'] = (msp_pre_post['post_freq'] - msp_pre_post['pre_freq']) / msp_pre_post['pre_freq']

Selecting to top 20 MSP codes with the largest average decrease in frequency post a Corrosion repair

In [56]:
msp_preds = msp_pre_post[msp_pre_post['Corrosion'] == True].groupby('MSP').agg({'per_change': 'mean'}).sort_values('per_change').head(20).index

Filtering MSP codes of interest

In [65]:
msp_corr_pre = msp_corr_pre[msp_corr_pre['MSP'].isin(msp_preds)].reset_index(drop=True)

In [67]:
msp_corr_pre.head()

,Aircraft,Repair Date,MSP,Corrosion,pre_freq
0,42,2012-04-26,FVXUMQ,False,1
1,43,2015-05-06,MZZJHP,False,2
2,43,2015-05-07,MZZJHP,False,2
3,43,2015-05-08,MZZJHP,False,2
4,43,2015-05-09,MZZJHP,False,2


In [69]:
msp_corr_pre = msp_corr_pre.drop_duplicates(['Aircraft', 'Repair Date', 'Corrosion'])

In [70]:
msp_prediction = msp_corr_pre.pivot_table(index = ['Aircraft', 'Repair Date', 'Corrosion'], columns = 'MSP', values = 'pre_freq', fill_value = 0).reset_index()

Save MSP count file to csv

In [73]:
msp_prediction.to_csv('../../HTM_data/htm_predict.csv', index = False)

Selecting the first 25 aircrafts to train, last 20 to test

In [107]:
train_data = msp_prediction[msp_prediction['Aircraft'].isin(range(26))]

x_train = train_data.drop(['Aircraft', 'Repair Date', 'Corrosion'], axis=1)
y_train = train_data['Corrosion']

test_data = msp_prediction[msp_prediction['Aircraft'].isin(range(26,46))]

x_test = test_data.drop(['Aircraft', 'Repair Date', 'Corrosion'], axis=1)
y_test = test_data['Corrosion']

### Ridge Logistic Regression Model

In [108]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import confusion_matrix, accuracy_score

# ignore convergance warnings
from warnings import filterwarnings
filterwarnings('ignore')

Fitting model 

In [113]:
ridge = LogisticRegressionCV(penalty='l2', class_weight='balanced', cv = 5, max_iter=2500).fit(x_train, y_train)

In [114]:
accuracy_score(y_test, ridge.predict(x_test)).round(3)

0.449

In [115]:
confusion_matrix(y_test, ridge.predict(x_test))

array([[206, 255],
       [  5,   6]])

### XGBoost

In [128]:
from sklearn.ensemble import GradientBoostingClassifier

In [155]:
xgb = GradientBoostingClassifier(loss = 'deviance', n_estimators=10000, learning_rate=0.01, max_depth=10).fit(x_train, y_train)

In [156]:
accuracy_score(y_test, xgb.predict(x_test)).round(3)

0.977

In [157]:
confusion_matrix(y_test, xgb.predict(x_test))

array([[461,   0],
       [ 11,   0]])

Given the XGBoost classifier is classifying only negative cases, we will need to upsample/downsample or investigate with MSP codes should be used for prediction

### Random Forest

In [116]:
from sklearn.ensemble import RandomForestRegressor

In [117]:
rf = RandomForestRegressor(n_estimators = 1000, random_state = 42).fit(x_train, y_train)

In [124]:
accuracy_score(y_test, rf.predict(x_test) > 0.1).round(3)

0.82

In [126]:
confusion_matrix(y_test, rf.predict(x_test) > 0.1)

array([[386,  75],
       [ 10,   1]])